In [ ]:
import pandas as pd 
import plotly.express as px8
from docx.shared import Cm
from docxtpl import DocxTemplate, InlineImage
from docx.shared import Cm, Inches, Mm, Emu
import random
import time
timestr = time.strftime("%Y-%m-%d")
df = pd.read_excel('C:/Users/152681/Downloads/AUTO_REPORT/SML//SML_Report_Data.xlsx')

Driving = df[(df.Ignition == 1)]
Charging = df[(df.Lock_Status == 1)]


EnergyA = (Driving.BattA_Voltage * Driving.BattA_Current)/(3600*1000)
EnergyB = (Driving.BattB_voltage * Driving.BattB_Current)/(3600*1000)
Energy = ((max(Driving.BattA_Voltage,Driving.BattB_voltage)) * (Driving.BattA_Current + Driving.BattB_Current))/(3600*1000)

DCDC_KWH = (Driving.DC_DC_voltage * Driving.DC_DC_Current)/(3600*1000)
EAC_KWH = (Driving.EAC_voltage * Driving.EAC_Current)/(3600*1000)
EHPS_KWH = (Driving.EHPS_voltage * Driving.EHPS_Current)/(3600*1000)
BCS_KWH = (Driving.BCS_voltage * Driving.BCS_Current)/(3600*1000)
MCU_KWH = (Driving.MCU_voltage * Driving.MCU_Current)/(3600*1000)
AC_KWH = (Driving.AC_voltage * Driving.AC_Current)/(3600*1000)
Driving = Driving.assign(Energy=Energy,DCDC_KWH=DCDC_KWH,EAC_KWH=EAC_KWH,EHPS_KWH=EHPS_KWH,BCS_KWH=BCS_KWH,
                         MCU_KWH=MCU_KWH,AC_KWH=AC_KWH)

EnergyUsedDriving = DCDC_KWH + EAC_KWH + EHPS_KWH + BCS_KWH + MCU_KWH + AC_KWH

EnergyChargingA = (Charging.BattA_Voltage * Charging.BattA_Current)/(3600*1000)
EnergyChargingB = (Charging.BattB_voltage * Charging.BattB_Current)/(3600*1000)
EnergyCharging = ((max(Charging.BattA_Voltage,Charging.BattB_voltage)) * (Charging.BattA_Current + Charging.BattB_Current))/(3600*1000)

DCDC_KWHCharging = (Charging.DC_DC_voltage * Charging.DC_DC_voltage)/(3600*1000)
BCS_KWH = (Charging.BCS_voltage * Charging.BCS_Current)/(3600*1000)
Charging = Charging.assign(EnergyCharging=EnergyCharging,DCDC_KWHCharging=DCDC_KWHCharging,BCS_KWH =BCS_KWH)

EnergyUsedCharging = DCDC_KWHCharging + BCS_KWH


Running = Driving[(Driving.State_Flag > 31)]
Running_Forward = Driving[(Driving.State_Flag == 32) & (Driving.Mot_Speed > 0)]
Running_regen = Running[(Running.Batt_Amp > 0)]
Running_mot = Running[(Running.Batt_Amp < 0)]

Total_KWH_Charge =round(sum(Charging.EnergyCharging),3)
Total_KWH_Motoring =round((sum(Driving.Energy) * -1),3)
Total_KWH_Regen =round(sum(Running_regen.Energy),3)
DCDC_KWH_Running =round(sum(Running.DCDC_KWH),3)
DCDC_KWH_Charging =round(sum(Charging.DCDC_KWHCharging),3)

veh_speed = Driving.Vehicle_Speed
Distance = round(sum(Running.veh_speed / 3600),3)
Net_Energy_Running = round((Total_KWH_Motoring + Total_KWH_Regen),3)
KWH_Km = round((Net_Energy_Running / Distance),3)

Start_SOC_Charging = round(min(Charging.Batt_SOC,default=0),3)
End_SOC_Charging = round(max(Charging.Batt_SOC,default=0),3)
Start_SOC_Running = round(max(Running.Batt_SOC,default=0),3)
End_SOC_Running = round(min(Running.Batt_SOC,default=0),3)
SOC_consumed = round((Start_SOC_Running - End_SOC_Running),3)
KWH_SOC_running = round((SOC_consumed * 9.2/100),3)
SOC_Charged = round((End_SOC_Charging - Start_SOC_Charging),3)
KWH_Charged = round((SOC_Charged*9.2/100),3)
Net_Energy_Charged = round((KWH_Charged - DCDC_KWH_Charging),3)

Max_motor_Temp = round(max(Running.MCU_Temp),3)
Min_motor_Temp = round(min(Running.MCU_Temp),3)
Max_Inv_Temp = round(max(Running.MCU_INV_Temp),3)
Min_Inv_Temp = round(min(Running.MCU_INV_Temp),3)
Max_EAC_Temp = round(max(Running.EAC_Temp),3)
Min_EAC_Temp = round(min(Running.EAC_Temp),3)
Max_EAC_INV_Temp = round(max(Running.EAC_INV_Temp),3)
Min_EAC_INV_Temp = round(min(Running.EAC_INV_Temp),3)
Max_EHPS_Temp = round(max(Running.EHPS_Temp),3)
Min_EHPS_Temp = round(min(Running.EHPS_Temp),3)
Max_EHPS_INV_Temp = round(max(Running.EHPS_INV_Temp),3)
Min_EHPS_INV_Temp = round(min(Running.EHPS_INV_Temp),3)
Max_DCDC_Temp = round(max(Running.DCDC_Temp),3)
Min_DCDC_Temp = round(min(Running.DCDC_Temp),3)
Max_TCS_Temp = round(max(Running.TCS_Temp),3)
Min_TCS_Temp = round(min(Running.TCS_Temp),3)

# Graph for Battery amp vs time
fig_Batt_Amp = make_subplots(specs=[[{"secondary_y": True}]])
fig_Batt_Amp.add_trace(go.Scatter(x=Driving.Time, y=Driving.BattA_Current, name="BattA<br>Curr(Amp)"),secondary_y=False,)
fig_Batt_Amp.add_trace(go.Scatter(x=Driving.Time, y=Driving.BattB_Current, name="BattB<br>Curr(Amp)"),secondary_y=True,)
fig_Batt_Amp.update_layout(title_text="BattA_Current & BattB_Current Vs Time")
fig_Batt_Amp.update_xaxes(title_text="Time (s)")
fig_Batt_Amp.update_yaxes(title_text="BattA_Current (Amp)", secondary_y=False)
fig_Batt_Amp.update_yaxes(title_text="BattB_Current (Amp)", secondary_y=True)
fig_Batt_Amp.write_image('fig_Batt_Amp.png')

# Graph for vehicle speed vs time
fig_veh_speed_Running = px.line(Running, x='Time',y = 'Vehicle_Speed', title='Vehicle Speed while Running')
fig_veh_speed_Running.update_layout(xaxis_title='Time(s)',yaxis_title='Vehicle Speed (Kmph)')
fig_veh_speed_Running.write_image('fig_VehicleSpeed_Running.png')

# Graph for Battery temperature vs SOC
fig_Cell_T_Max_SOC = make_subplots(specs=[[{"secondary_y": True}]])
fig_Cell_T_Max_SOC.add_trace(go.Scatter(x=Driving.Time, y=Driving.Batt_Temp, name="Batt<br>Temp(DegC)"),secondary_y=False,)
fig_Cell_T_Max_SOC.add_trace(go.Scatter(x=Driving.Time, y=Driving.Batt_SOC, name="Battery<br>SOC(%)"),secondary_y=True,)
fig_Cell_T_Max_SOC.update_layout(title_text="Batt Temperature & Battery SOC Vs Time")
fig_Cell_T_Max_SOC.update_xaxes(title_text="Time (s)")
fig_Cell_T_Max_SOC.update_yaxes(title_text="Batt Temperature (DegC)", secondary_y=False)
fig_Cell_T_Max_SOC.update_yaxes(title_text="Battery SOC (%)", secondary_y=True)
fig_Cell_T_Max_SOC.write_image('fig_Batt_T_Max_SOC.png')

# Gragh for inverter temperature vs SOC
fig_Inv_Temp_SOC = make_subplots(specs=[[{"secondary_y": True}]])
fig_Inv_Temp_SOC.add_trace(go.Scatter(x=Driving.Time, y=Driving.MCU_INV_Temp, name="Motor<br>Controller<br>Temp(DegC)"),secondary_y=False,)
fig_Inv_Temp_SOC.add_trace(go.Scatter(x=Driving.Time, y=Driving.Batt_SOC, name="Battery<br>SOC(%)"),secondary_y=True,)
fig_Inv_Temp_SOC.update_layout(title_text="Motor Controller Temperature & Battery SOC Vs Time")
fig_Inv_Temp_SOC.update_xaxes(title_text="Time (s)")
fig_Inv_Temp_SOC.update_yaxes(title_text="Motor Controller Temperature (DegC)", secondary_y=False)
fig_Inv_Temp_SOC.update_yaxes(title_text="Battery SOC (%)", secondary_y=True)
fig_Inv_Temp_SOC.write_image('fig_Inv_Temp_SOC.png')

#Graph for Motor temperature vs SOC
fig_Mot_Temp_SOC = make_subplots(specs=[[{"secondary_y": True}]])
fig_Mot_Temp_SOC.add_trace(go.Scatter(x=Driving.Time, y=Driving.MCU_Temp, name="Motor<br>Temp(DegC)"),secondary_y=False,)
fig_Mot_Temp_SOC.add_trace(go.Scatter(x=Driving.Time, y=Driving.Batt_SOC, name="Battery<br>SOC(%)"),secondary_y=True,)
fig_Mot_Temp_SOC.update_layout(title_text="Motor Temperature & Battery SOC Vs Time")
fig_Mot_Temp_SOC.update_xaxes(title_text="Time (s)")
fig_Mot_Temp_SOC.update_yaxes(title_text="Motor Temperature (DegC)", secondary_y=False)
fig_Mot_Temp_SOC.update_yaxes(title_text="Battery SOC (%)", secondary_y=True)
fig_Mot_Temp_SOC.write_image('fig_Mot_Temp_SOC.png')

#Graph for Motor controller temperature vs Motor Temperature
fig_Inv_Temp_Mot_Temp = make_subplots(specs=[[{"secondary_y": True}]])
fig_Inv_Temp_Mot_Temp.add_trace(go.Scatter(x=Driving.Time, y=Driving.MCU_INV_Temp, name="Motor<br>Controller<br>Temp(DegC)"),secondary_y=False,)
fig_Inv_Temp_Mot_Temp.add_trace(go.Scatter(x=Driving.Time, y=Driving.MCU_Temp, name="Motor<br>Temp(DegC)"),secondary_y=True,)
fig_Inv_Temp_Mot_Temp.update_layout(title_text="Motor Controller Temperature & Motor Tempreture Vs Time")
fig_Inv_Temp_Mot_Temp.update_xaxes(title_text="Time (s)")
fig_Inv_Temp_Mot_Temp.update_yaxes(title_text="Motor Controller Temperature (DegC)", secondary_y=False)
fig_Inv_Temp_Mot_Temp.update_yaxes(title_text="Motor Tempreture(DegC)", secondary_y=True)
fig_Inv_Temp_Mot_Temp.write_image('fig_Inv_Temp_Mot_Temp.png')

# Graph for vehicle speed vs time (Histogram)
fig_veh_speed_Running_Hist = px.histogram(Running_Forward, x = 'veh_speed',y='Time' ,title='Vehicle Speed While Running',range_x=[1, 60],histnorm='percent',nbins = 20,text_auto=True)
fig_veh_speed_Running_Hist.update_layout(xaxis_title='Vehicle Speed (Kmph)',yaxis_title='Count(%)')
fig_veh_speed_Running_Hist.update_traces(texttemplate='%{y:.2f}')
fig_veh_speed_Running_Hist.update_xaxes(showgrid=True,gridwidth=1, gridcolor='White')
fig_veh_speed_Running_Hist.update_yaxes(showgrid=True,gridwidth=1, gridcolor='White')
fig_veh_speed_Running_Hist.write_image('fig_veh_speed_Running_Hist.png')

# Graph for Motor speed vs time (Histogram)
fig_Mot_Speed_Hist = px.histogram(Running_Forward, x = 'Mot_Speed',y='Time',range_x=[0, 7000], title='Motor Speed While Running',histnorm='percent',nbins = 20,text_auto=True)
fig_Mot_Speed_Hist.update_layout(xaxis_title='Motor Speed (RPM)',yaxis_title='Count(%)')
fig_Mot_Speed_Hist.update_traces(texttemplate='%{y:.2f}')
fig_Mot_Speed_Hist.update_xaxes(showgrid=True,gridwidth=1, gridcolor='White')
fig_Mot_Speed_Hist.update_yaxes(showgrid=True,gridwidth=1, gridcolor='White')
fig_Mot_Speed_Hist.write_image('fig_Mot_Speed_Hist.png')

#Graph for Motor torque vs Time
fig_Current_Torque_of_M_Hist = px.histogram(Running_Forward, x = 'Mot_Torque',y='Time', range_x=[1, 100],title='Motor Torque',nbins = 30)
fig_Current_Torque_of_M_Hist.update_layout(xaxis_title='Motor Torque (Nm)',yaxis_title='Count(%)')
fig_Current_Torque_of_M_Hist.write_image('fig_Current_Torque_of_M_Hist.png')

#Graph for Motor Torque vs Motor Speed
fig_Current_Torque_of_M_Mot_Speed = px.scatter(Running, x = 'Mot_Speed', y = 'Mot_Torque', title='Motor Torque Vs Speed')
fig_Current_Torque_of_M_Mot_Speed.update_layout(xaxis_title='Motor Speed (RPM)',yaxis_title='Motor Torque(Nm)')
fig_Current_Torque_of_M_Mot_Speed.write_image('fig_Current_Torque_of_M_Mot_Speed.png')
# Write data into word file
template = DocxTemplate(r"C:\Users\152681\Downloads\AUTO_REPORT\EV_Report_Copy.docx")
context = {
    'timestr': timestr,
    'Total_KWH_Charge': Total_KWH_Charge,
    'Total_KWH_Motoring': Total_KWH_Motoring,
    'Total_KWH_Regen': Total_KWH_Regen,
    'DCDC_KWH_Running': DCDC_KWH_Running,
    'DCDC_KWH_Charging': DCDC_KWH_Charging,
    'Distance': Distance,
    'Net_Energy_Running': Net_Energy_Running,
    'KWH_Km': KWH_Km,
    'Start_SOC_Charging': Start_SOC_Charging,
    'End_SOC_Charging': End_SOC_Charging,
    'Start_SOC_Running' : Start_SOC_Running,
    'End_SOC_Running' : End_SOC_Running,
    'Max_motor_Temp': Max_motor_Temp,
    'Max_Inv_Temp': Max_Inv_Temp,
    'Min_Inv_Temp': Min_Inv_Temp,
    'Max_Cell_Temp' : Max_Cell_Temp,
    'SOC_consumed': SOC_consumed,
    'KWH_Charged': KWH_Charged,
    'Net_Energy_Charged': Net_Energy_Charged,
    'Net_Energy_Running': Net_Energy_Running,
    'fig_Batt_Amp': InlineImage(template,'fig_Batt_Amp.png',Cm(20)),
    'fig_VehicleSpeed_Running':InlineImage(template,'fig_VehicleSpeed_Running.png',Cm(20)),
    'fig_Cell_T_Max_SOC':InlineImage(template,'fig_Cell_T_Max_SOC.png',Cm(20)),
    'fig_Inv_Temp_SOC':InlineImage(template,'fig_Inv_Temp_SOC.png',Cm(20)),
    'fig_Mot_Temp_SOC':InlineImage(template,'fig_Mot_Temp_SOC.png',Cm(20)),
    'fig_Inv_Temp_Mot_Temp':InlineImage(template,'fig_Inv_Temp_Mot_Temp.png',Cm(20)),
    'fig_veh_speed_Running_Hist':InlineImage(template,'fig_veh_speed_Running_Hist.png',Cm(20)),
    'fig_Mot_Speed_Hist':InlineImage(template,'fig_Mot_Speed_Hist.png',Cm(20)),
    'fig_Current_Torque_of_M_Hist':InlineImage(template,'fig_Current_Torque_of_M_Hist.png',Cm(20)),
    'fig_Current_Torque_of_M_Mot_Speed':InlineImage(template,'fig_Current_Torque_of_M_Mot_Speed.png',Cm(20)),  
    }
template.render(context)
template.save('SML_Report.docx')
print("Done - Now check your files")

In [108]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.graph_objs as go

# Sample data (replace with your own data)
df = pd.read_excel('C:/Users/152681/Downloads/AUTO_REPORT/SML//3_GOLD.xlsx')

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the layout of the Dash app
app.layout = html.Div([
    html.H1("SML_Dashboard",style={
            'textAlign': 'center',
        }),
    
    html.Label('Select Column:'),
    dcc.Dropdown(
        id='dropdown-column',
        options=[{'label': col, 'value': col} for col in df.columns[1:]],
        value=df.columns[1],
        clearable=False,style={
            'textAlign': 'center',
            'color': colors['text'],'width': '48%', 'float': 'Center', 'display': 'inline-block'}
    ),
    
    dcc.Graph(id='time-series-graph')
])

# Define callback to update the graph based on dropdown selection
@app.callback(
    Output('time-series-graph', 'figure'),
    [Input('dropdown-column', 'value')]
)
def update_graph(selected_column):
    trace = go.Scatter(x=df['Time'], y=df[selected_column], mode='lines+markers', name=selected_column)
    layout = go.Layout(title='Time Series Graph', xaxis=dict(title='Time'), yaxis=dict(title=selected_column))
    return {'data': [trace], 'layout': layout}

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)

Dash app running on http://127.0.0.1:8050/
